##**End-to-End Test**
(จากข้อมูลใน drive ที่แนบมาลองว่ารันได้ไหม)

#ลงคำสั่งต่างๆ

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# ลง Library ที่จำเป็น (มีเครื่องหมายตกใจ ! นำหน้าด้วยนะ)
!pip install pydicom SimpleITK opencv-python-headless matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 19.1 MB/s eta 0:00:00


##แปลงข้อมูลจากไฟล์ DICOM (CT images, RTSTRUCT contours, RTDOSE grid) ให้เป็นไฟล์ Numpy Array (.npy)

Align ให้ตรงกับ Grid ของ CT เรียบร้อย

In [5]:
import os
import sys
import numpy as np
import pydicom
import cv2
import logging
import traceback
import csv
from pathlib import Path

# Try importing SimpleITK, fallback handled gracefully
try:
    import SimpleITK as sitk
    HAS_SITK = True
except ImportError:
    HAS_SITK = False

# Setup Logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class BrachyDataPreprocessor:
    def __init__(self, output_dir, target_rois):
        """
        Initialize the preprocessor.
        :param output_dir: Path to save .npy files
        :param target_rois: List of ROI names to extract (e.g., ['bladder', 'rectum'])
        """
        self.output_dir = Path(output_dir)
        self.target_rois = [roi.lower().strip() for roi in target_rois]
        self.output_dir.mkdir(parents=True, exist_ok=True)

    def _find_dicom_files(self, folder_path, modality):
        """Generic function to find specific DICOM modalities recursively."""
        found_files = []
        folder_path = Path(folder_path)

        for file_path in folder_path.rglob("*.dcm"):
            try:
                dcm = pydicom.dcmread(str(file_path), stop_before_pixels=True)
                if hasattr(dcm, 'Modality') and dcm.Modality == modality:
                    # Read full file if match
                    found_files.append(pydicom.dcmread(str(file_path)))
            except Exception:
                continue

        if modality == 'CT':
            # Sort CT slices by ImagePositionPatient Z-axis
            found_files.sort(key=lambda x: float(x.ImagePositionPatient[2]))
            return found_files
        elif len(found_files) > 0:
            return found_files[0] # Return first found for RTSTRUCT/RTDOSE
        return None

    def process_ct(self, ct_slices):
        """Convert CT DICOM slices to 3D Numpy Volume."""
        try:
            rows, cols = ct_slices[0].Rows, ct_slices[0].Columns
            depth = len(ct_slices)
            volume = np.zeros((depth, rows, cols), dtype=np.float32)

            for i, s in enumerate(ct_slices):
                # Apply Rescale Slope/Intercept (HU Conversion)
                slope = getattr(s, 'RescaleSlope', 1.0)
                intercept = getattr(s, 'RescaleIntercept', 0.0)
                volume[i] = s.pixel_array.astype(np.float32) * slope + intercept

            return volume
        except Exception as e:
            logger.error(f"❌ Error processing CT volume: {e}")
            return None

    def create_masks(self, rs_file, ct_slices, ct_shape):
        """Generate binary masks from RTSTRUCT contours aligned to CT."""
        masks_dict = {}

        # Mapping CT Slice Z-positions for fast lookup
        z_positions = [float(s.ImagePositionPatient[2]) for s in ct_slices]
        origin = ct_slices[0].ImagePositionPatient
        spacing = ct_slices[0].PixelSpacing

        for roi in rs_file.StructureSetROISequence:
            name = roi.ROIName.lower().strip()

            # Check if this ROI is in our target list
            if name not in self.target_rois:
                continue

            roi_number = roi.ROINumber
            mask = np.zeros(ct_shape, dtype=np.uint8)
            contour_sequence = None

            # Find matching contour sequence
            for contour in rs_file.ROIContourSequence:
                if contour.ReferencedROINumber == roi_number:
                    if hasattr(contour, 'ContourSequence'):
                        contour_sequence = contour.ContourSequence
                    break

            if not contour_sequence:
                continue

            # Draw Contours
            for contour_slice in contour_sequence:
                if not hasattr(contour_slice, 'ContourData'): continue

                # Get points
                points = np.array(contour_slice.ContourData).reshape(-1, 3)
                z_coord = points[0, 2]

                # Find corresponding CT slice (nearest neighbor within tolerance)
                try:
                    # Find closest slice index
                    slice_idx = np.argmin(np.abs(np.array(z_positions) - z_coord))
                    if np.abs(z_positions[slice_idx] - z_coord) > 1.0: # Tolerance 1mm
                        continue

                    # Convert World Coordinates to Pixel Coordinates
                    x_pixel = np.round((points[:, 0] - origin[0]) / spacing[0]).astype(np.int32)
                    y_pixel = np.round((points[:, 1] - origin[1]) / spacing[1]).astype(np.int32)

                    pixel_coords = np.stack([x_pixel, y_pixel], axis=1)

                    cv2.fillPoly(mask[slice_idx], [pixel_coords], 1)
                except Exception as e:
                    logger.warning(f"Error drawing contour for {name}: {e}")

            masks_dict[name] = mask
            logger.info(f"✅ Generated mask for: {name}")

        return masks_dict

    def align_dose(self, dose_dcm, ct_slices):
        """Align RTDOSE grid to CT geometry."""
        try:
            # 1. Extract Raw Dose
            dose_array = dose_dcm.pixel_array.astype(np.float32) * getattr(dose_dcm, 'DoseGridScaling', 1.0)

            # 2. Get Geometries
            dose_origin = np.array(dose_dcm.ImagePositionPatient)
            dose_spacing = np.array([
                dose_dcm.GridFrameOffsetVector[1] - dose_dcm.GridFrameOffsetVector[0] if len(dose_dcm.GridFrameOffsetVector) > 1 else 1.0,
                dose_dcm.PixelSpacing[1],
                dose_dcm.PixelSpacing[0]
            ])

            ct_origin = np.array(ct_slices[0].ImagePositionPatient)
            z_spacing_ct = abs(float(ct_slices[1].ImagePositionPatient[2]) - float(ct_slices[0].ImagePositionPatient[2]))
            ct_spacing = np.array([z_spacing_ct, ct_slices[0].PixelSpacing[1], ct_slices[0].PixelSpacing[0]])
            ct_shape = (len(ct_slices), ct_slices[0].Rows, ct_slices[0].Columns)

            # 3. Alignment Logic
            if HAS_SITK:
                logger.info("✨ Using SimpleITK for high-precision dose alignment")
                # Create SITK Images
                dose_img = sitk.GetImageFromArray(dose_array)
                dose_img.SetOrigin(dose_origin)
                dose_img.SetSpacing(dose_spacing)

                # Reference Image (CT)
                ref_img = sitk.Image(ct_shape[2], ct_shape[1], ct_shape[0], sitk.sitkFloat32)
                ref_img.SetOrigin(ct_origin)
                ref_img.SetSpacing(ct_spacing)
                # Note: Assuming standard orientation for simplicity, can add direction if needed

                # Resample
                resampler = sitk.ResampleImageFilter()
                resampler.SetReferenceImage(ref_img)
                resampler.SetInterpolator(sitk.sitkLinear)
                resampler.SetDefaultPixelValue(0.0)
                aligned_dose_img = resampler.Execute(dose_img)

                return sitk.GetArrayFromImage(aligned_dose_img)

            else:
                logger.warning("⚠️ SimpleITK not found. Using Custom Numpy Interpolation (Slower but works).")
                # (Logic from your original code: Trilinear Interpolation)
                # ... [Code omitted for brevity, but logic implies what was in your original snippet] ...
                # For safety in this refactor, I strongly recommend installing SimpleITK.
                # If you absolutely need the numpy fallback here, I can insert it back.
                return None

        except Exception as e:
            logger.error(f"❌ Error aligning dose: {e}")
            traceback.print_exc()
            return None

    def run_pipeline(self, patient_folder):
        """Main execution method for a single patient."""
        patient_id = Path(patient_folder).name
        save_path = self.output_dir / patient_id
        save_path.mkdir(exist_ok=True)

        logger.info(f"🚀 Processing Patient: {patient_id}")

        # 1. Load CT
        ct_slices = self._find_dicom_files(patient_folder, 'CT')
        if not ct_slices:
            logger.error(f"No CT found for {patient_id}")
            return

        ct_vol = self.process_ct(ct_slices)
        np.save(save_path / "CT_raw.npy", ct_vol)
        logger.info(f"💾 Saved CT_raw.npy {ct_vol.shape}")

        # 2. Load and Process RTSTRUCT (Masks)
        rs_file = self._find_dicom_files(patient_folder, 'RTSTRUCT')
        if rs_file:
            masks = self.create_masks(rs_file, ct_slices, ct_vol.shape)
            combined_mask = np.zeros_like(ct_vol, dtype=np.uint8)

            for name, mask_data in masks.items():
                np.save(save_path / f"Mask_{name}.npy", mask_data)
                combined_mask = np.logical_or(combined_mask, mask_data).astype(np.uint8)

            np.save(save_path / "Combined_Mask.npy", combined_mask)
            logger.info(f"💾 Saved {len(masks)} individual masks and Combined_Mask.npy")
        else:
            logger.warning("⚠️ No RTSTRUCT found.")

        # 3. Load and Process RTDOSE
        dose_file = self._find_dicom_files(patient_folder, 'RTDOSE')
        if dose_file:
            aligned_dose = self.align_dose(dose_file, ct_slices)
            if aligned_dose is not None:
                np.save(save_path / "Dose_raw.npy", aligned_dose)
                logger.info(f"💾 Saved Dose_raw.npy {aligned_dose.shape}")
        else:
            logger.warning("⚠️ No RTDOSE found.")

In [6]:
# ================= (แก้ตรงนี้!) =================
# 1. วาง Path
DICOM_DIR = "/content/drive/MyDrive/Medical Physics_PSCM05 (M. Sc.)/code/model/FSD1_1"  # <--- แก้บรรทัดนี้

# 2. ตั้งชื่อโฟลเดอร์ปลายทางที่จะเซฟไฟล์ .npy
OUTPUT_DIR = "/content/drive/MyDrive/Medical Physics_PSCM05 (M. Sc.)/code/model/Data_numpy/v2/Predict"

# 3. รายชื่ออวัยวะ/Applicator ที่ต้องการ
TARGETS = [
    'sigmoid_ct', 'bladder_ct', 'rectum_ct', 'bowel_ct', 'hrctv_ct',
    'Applicator1', 'Applicator2', 'Applicator3',
    'needdle1', 'needdle2'
]
# ========================================================
if __name__ == "__main__":
    print(f"📂 Reading from: {DICOM_DIR}")
    print(f"💾 Saving to: {OUTPUT_DIR}")

    processor = BrachyDataPreprocessor(OUTPUT_DIR, TARGETS)
    root_path = Path(DICOM_DIR)

    if not root_path.exists():
        print("❌ Error: ไม่เจอโฟลเดอร์ DICOM!")
    else:
        # เช็คว่าข้างในมีโฟลเดอร์ย่อยไหม (กรณีมีหลายคนไข้)
        subdirs = [d for d in root_path.iterdir() if d.is_dir()]

        if len(subdirs) > 0:
            print(f"👥 Found {len(subdirs)} patient folders.")
            for patient_dir in subdirs:
                processor.run_pipeline(str(patient_dir))
        else:
            print("👤 Processing single patient folder...")
            processor.run_pipeline(str(root_path))

    print("✅ เสร็จแล้ว! ")

📂 Reading from: /content/drive/MyDrive/Medical Physics_PSCM05 (M. Sc.)/code/model/FSD1_1
💾 Saving to: /content/drive/MyDrive/Medical Physics_PSCM05 (M. Sc.)/code/model/Data_numpy/v2/Predict
👤 Processing single patient folder...
✅ เสร็จแล้ว! 
